<a href="https://colab.research.google.com/github/joker2017/Calculator/blob/master/Mobilenet_v2_KKKKKKKKKKKKKKKKKKK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/tensorflow/models

fatal: destination path 'models' already exists and is not an empty directory.


In [0]:
import sys
sys.path.append('/content/models/research/slim')

In [3]:
import os
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
print( os.listdir('/content') )

Saving imagenet_class_names.txt to imagenet_class_names (1).txt
User uploaded file "imagenet_class_names.txt" with length 31674 bytes
['.config', 'mobilenet_v2_1', 'imagenet_class_names (1).txt', 'flower_photos', 'models', 'imagenet_class_names.txt', 'sample_data']


In [4]:


#Загрузите последнюю предварительную модель Inception v3: контрольная точка доступна по адресу https://github.com/tensorflow/models/tree/master/research/slim . 
#писок имен классов доступен по адресу https://goo.gl/brXRtZ , но вы должны сначала вставить «background» класс.
import os
import re 
import sys
import tarfile
import numpy as np
import tensorflow as tf
from random import sample
from six.moves import urllib
import matplotlib.image as mpimg
import tensorflow.contrib.slim as slim
#from tensorflow.contrib.slim.nets import inception
from nets.mobilenet import mobilenet_v2
from skimage.transform import rescale, resize, downscale_local_mean

MODEL_NAME = "mobilenet_v2_1.0_224.ckpt"
CLASS_NAME_REGEX = re.compile(r"^n\d+\s+(.*)\s*$", re.M | re.U)
DATASET_URL = "http://download.tensorflow.org/example_images/flower_photos.tgz"
MODEL_URL = "https://storage.googleapis.com/mobilenet_v2/checkpoints/mobilenet_v2_1.0_224.tgz"



def download_progress(count, block_size, total_size):
    percent = count * block_size * 100 // total_size
    sys.stdout.write("\rDownloading: {}%".format(percent))
    sys.stdout.flush()
    

def download_tgz(url):
    file_name=os.path.basename(url)
    name_parts = os.path.split(file_name)
    _, name = name_parts
    name_path = name.rsplit( ".", 2 )[ 0 ]
    path = os.path.join("/content", name_path)
    tgz_path = os.path.join(path, name)
    if os.path.exists(path): #and os.path.isfile(tgz_path):
      print("file and path exist ", tgz_path, "Path is:", os.listdir(path), "!")
      return path
    else:
      print(os.path.exists(path), os.path.isfile(tgz_path), path, tgz_path)
      os.makedirs(path, exist_ok=True)
      urllib.request.urlretrieve(url, tgz_path, reporthook=download_progress)
      tmp_tgz = tarfile.open(tgz_path)
      tmp_tgz.extractall(path=path)
      tmp_tgz.close()
      os.remove(tgz_path)
      print("path created:", path)
      return path
    
# Загружаем модель
MODELS_PATH = download_tgz(MODEL_URL)
# Загружаем датасет
FLOWERS_PATH = download_tgz(DATASET_URL)
 

def load_class_names():
    with open(os.path.join("/content", "imagenet_class_names.txt"), "rb") as f:
        content = f.read().decode("utf-8")
        #sys.stdout.flush()
        return CLASS_NAME_REGEX.findall(content)
      
load_class_names()    
class_names = ["background"] + load_class_names()       


file and path exist  /content/mobilenet_v2_1/mobilenet_v2_1.0_224.tgz Path is: ['mobilenet_v2_1.0_224.ckpt.meta', 'mobilenet_v2_1.0_224.tflite', 'mobilenet_v2_1.0_224.ckpt.index', 'mobilenet_v2_1.0_224_info.txt', 'mobilenet_v2_1.0_224_frozen.pb', 'mobilenet_v2_1.0_224_eval.pbtxt', 'mobilenet_v2_1.0_224.ckpt.data-00000-of-00001'] !
file and path exist  /content/flower_photos/flower_photos.tgz Path is: ['flower_photos'] !


In [54]:
kernel_size[0]

NameError: ignored

In [59]:
5//1

5

In [0]:

import contextlib
from functools import partial
def _fixed_padding(inputs, kernel_size, rate=1):     
  print(kernel_size)
  kernel_size_effective = [kernel_size[0] + (kernel_size[0] - 1) * (rate - 1),
                           kernel_size[0] + (kernel_size[0] - 1) * (rate - 1)]
  pad_total = [kernel_size_effective[0] - 1, kernel_size_effective[1] - 1]
  pad_beg = [pad_total[0] // 2, pad_total[1] // 2]
  pad_end = [pad_total[0] - pad_beg[0], pad_total[1] - pad_beg[1]]
  padded_inputs = tf.pad(inputs, [[0, 0], [pad_beg[0], pad_end[0]],
                                  [pad_beg[1], pad_end[1]], [0, 0]])
  return padded_inputs


def _make_divisible(v, divisor, min_value=None):
  if min_value is None:
    min_value = divisor
  new_v = max(min_value, int(v + divisor / 2) / divisor * divisor)
  # Make sure that round down does not go down by more than 10%.
  if new_v < 0.9 * v:
    new_v += divisor
  return new_v

def expand_input_by_factor(n, divisible_by=8):                                     ##
  return lambda num_inputs, **_: _make_divisible(num_inputs * n, divisible_by)



def expanded_conv(input_tensor):
  
    num_outputs=320
    expansion_size=expand_input_by_factor(6)
    stride=1,
    rate=1,
    kernel_size=(3, 3),
    residual=True,
    normalizer_fn=None,
    project_activation_fn=tf.identity,
    split_projection=1,
    split_expansion=1,
    expansion_transform=True,
    depthwise_location = 'output',
    depthwise_channel_multiplier=1,
    endpoints=None,
    use_explicit_padding=False,
    padding='SAME',
    scope=None
#  with tf.variable_scope(scope, default_name='expanded_conv') as s, \
#       tf.name_scope(s.original_name_scope):
    prev_depth = input_tensor.get_shape().as_list()[3]
    
    if use_explicit_padding:
      if padding != 'SAME':
        #raise TypeError('`use_explicit_padding` should only be used with '
                       # '"SAME" padding.')
        print("`use_explicit_padding` should only be used with  SAME padding.")
      padding = 'VALID'
    depthwise_func = partial(
        slim.separable_conv2d,
        num_outputs=None,
        kernel_size=kernel_size,
        depth_multiplier=depthwise_channel_multiplier,
        stride=stride,
        rate=rate,
        normalizer_fn=normalizer_fn,
        padding=padding,
        scope='depthwise')
    # b1 -> b2 * r -> b2
    #   i -> (o * r) (bottleneck) -> o
    input_tensor = tf.identity(input_tensor, 'input')
    net = input_tensor

    if depthwise_location == 'input':
      if use_explicit_padding:
        net = _fixed_padding(net, kernel_size, rate)
      net = depthwise_func(net, activation_fn=None)

    if callable(expansion_size):
      inner_size = expansion_size(num_inputs=prev_depth)
    else:
      inner_size = expansion_size

    if inner_size > net.shape[3]:
      net = split_conv(
          net,
          inner_size,
          num_ways=split_expansion,
          scope='expand',
          stride=1,
          normalizer_fn=normalizer_fn)
      net = tf.identity(net, 'expansion_output')
    if endpoints is not None:
      endpoints['expansion_output'] = net

    if depthwise_location == 'expansion':
      if use_explicit_padding:
        net = _fixed_padding(net, kernel_size, rate)
      net = depthwise_func(net)

    net = tf.identity(net, name='depthwise_output')
    if endpoints is not None:
      endpoints['depthwise_output'] = net
    if expansion_transform:
      net = expansion_transform(expansion_tensor=net, input_tensor=input_tensor)
    #Обратите внимание, что в отличие от расширения, у нас всегда есть проекция для получения желаемого выходного размера.
    net = split_conv(
        net,
        num_outputs,
        num_ways=split_projection,
        stride=1,
        scope='project',
        normalizer_fn=normalizer_fn,
        activation_fn=project_activation_fn)
    if endpoints is not None:
      endpoints['projection_output'] = net
    if depthwise_location == 'output':
      if use_explicit_padding:
        net = _fixed_padding(net, kernel_size, rate)
      net = depthwise_func(net, activation_fn=None)

    if callable(residual):  # custom residual/ таможенный остаток
      net = residual(input_tensor=input_tensor, output_tensor=net)
    elif (residual and
          # проверка шага гарантирует, что мы не добавляем остатки, когда пространственные измерения отсутствуют
          stride == 1 and
          # Depth matches/Глубина совпадений
          net.get_shape().as_list()[3] ==
          input_tensor.get_shape().as_list()[3]):
      net += input_tensor
    return tf.identity(net, name='output')


def split_conv(input_tensor,
               num_outputs,
               num_ways,
               scope,
               divisible_by=8,
               **kwargs):

  b = input_tensor.get_shape().as_list()[3]

  if num_ways == 1 or min(b / num_ways,
                          num_outputs / num_ways) < divisible_by:
    # Don't do any splitting if we end up with less than 8 filters
    # on either side.
    return slim.conv2d(input_tensor, num_outputs, [1, 1], scope=scope, **kwargs)

  outs = []
  input_splits = _split_divisible(b, num_ways, divisible_by=divisible_by)
  output_splits = _split_divisible(
      num_outputs, num_ways, divisible_by=divisible_by)
  inputs = tf.split(input_tensor, input_splits, axis=3, name='split_' + scope)
  base = scope
  for i, (input_tensor, out_size) in enumerate(zip(inputs, output_splits)):
    scope = base + '_part_%d' % (i,)
    n = slim.conv2d(input_tensor, out_size, [1, 1], scope=scope, **kwargs)
    n = tf.identity(n, scope + '_output')
    outs.append(n)
  return tf.concat(outs, 3, name=scope + '_concat')

In [55]:
kernel_size[0]

NameError: ignored

In [0]:
def global_pool(input_tensor, pool_op=tf.nn.avg_pool):

  shape = input_tensor.get_shape().as_list()
  if shape[1] is None or shape[2] is None:
    kernel_size = tf.convert_to_tensor(
        [1, tf.shape(input_tensor)[1],
         tf.shape(input_tensor)[2], 1])
  else:
    kernel_size = [1, shape[1], shape[2], 1]
  output = pool_op(
      input_tensor, ksize=kernel_size, strides=[1, 1, 1, 1], padding='VALID')
  # Recover output shape, for unknown shape.
  output.set_shape([None, 1, 1, None])
  return output

In [7]:
width = 224 #299
height = 224 #299
channels = 3
import tensorflow as tf
tf.reset_default_graph()
n_epochs = 1000
batch_size = 40
best_loss_val = np.infty
check_interval = 500
checks_since_last_progress = 0
max_checks_without_progress = 20
best_model_params = None 





#Каждый подкаталог flower_photos каталога содержит все изображения данного класса. Давайте получим список классов:
flowers_root_path = os.path.join(FLOWERS_PATH, "flower_photos")
flower_classes = sorted([dirname for dirname in os.listdir(flowers_root_path)
                  if os.path.isdir(os.path.join(flowers_root_path, dirname))])

#Давайте получим список всех путей файлов изображений для каждого класса:

from collections import defaultdict
image_paths = defaultdict(list)
for flower_class in flower_classes:
    image_dir = os.path.join(flowers_root_path, flower_class)
    for filepath in os.listdir(image_dir):
        if filepath.endswith(".jpg"):
            image_paths[flower_class].append(os.path.join(image_dir, filepath))
            
            
#Давайте сортируем пути изображения, чтобы заставить этот ноутбук вести себя последовательно на нескольких запусках:
for paths in image_paths.values():
    paths.sort() 

    
#Для получения дополнительных функций манипуляции с изображениями, таких как вращения, проверьте документацию SciPy или эту приятную страницу .
from scipy.misc import imresize
def prepare_image(image, target_width = width, target_height = height, max_zoom = 0.2):
    """Zooms and crops the image randomly for data augmentation."""

    # First, let's find the largest bounding box with the target size ratio that fits within the image
    height = image.shape[0]
    width = image.shape[1]
    image_ratio = width / height
    target_image_ratio = target_width / target_height
    crop_vertically = image_ratio < target_image_ratio
    crop_width = width if crop_vertically else int(height * target_image_ratio)
    crop_height = int(width / target_image_ratio) if crop_vertically else height
        
    # Now let's shrink this bounding box by a random factor (dividing the dimensions by a random number
    # between 1.0 and 1.0 + `max_zoom`.
    resize_factor = np.random.rand() * max_zoom + 1.0
    crop_width = int(crop_width / resize_factor)
    crop_height = int(crop_height / resize_factor)
    
    # Next, we can select a random location on the image for this bounding box.
    x0 = np.random.randint(0, width - crop_width)
    y0 = np.random.randint(0, height - crop_height)
    x1 = x0 + crop_width
    y1 = y0 + crop_height
    
    # Let's crop the image using the random bounding box we built.
    image = image[y0:y1, x0:x1]

    # Let's also flip the image horizontally with 50% probability:
    if np.random.rand() < 0.5:
        image = np.fliplr(image)

    # Now, let's resize the image to the target dimensions.
    image = imresize(image, (target_width, target_height))
    
    # Finally, let's ensure that the colors are represented as
    # 32-bit floats ranging from 0.0 to 1.0 (for now):
    return image.astype(np.float32) / 255


def prepare_batch(flower_paths_and_classes, batch_size):
    batch_paths_and_classes = sample(flower_paths_and_classes, batch_size)
    images = [mpimg.imread(path)[:, :, :channels] for path, labels in batch_paths_and_classes]
    prepared_images = [prepare_image(image) for image in images]
    X_batch = 2 * np.stack(prepared_images) - 1 # Inception expects colors ranging from -1 to 1
    y_batch = np.array([labels for path, labels in batch_paths_and_classes], dtype=np.int32)
    return X_batch, y_batch 
  
  
def prepare_data(flower_paths_and_classes):
    batch_paths_and_classes = flower_paths_and_classes
    images = [mpimg.imread(path)[:, :, :channels] for path, labels in batch_paths_and_classes]
    prepared_images = [prepare_image(image) for image in images]
    X_data = 2 * np.stack(prepared_images) - 1 # Inception expects colors ranging from -1 to 1
    y_data = np.array([labels for path, labels in batch_paths_and_classes], dtype=np.int32)
    return X_data, y_data  
  
  
#раннняя остановка
def get_model_params():
    gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    return {gvar.op.name: value for gvar, value in zip(gvars, tf.get_default_session().run(gvars))}

def restore_model_params(model_params):
    gvar_names = list(model_params.keys())
    assign_ops = {gvar_name: tf.get_default_graph().get_operation_by_name(gvar_name + "/Assign")
                  for gvar_name in gvar_names}
    init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
    feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
    tf.get_default_session().run(assign_ops, feed_dict=feed_dict)  
  
  #снова получить начальный график v3. На этот раз воспользуемся trainingзаполнителем
  
X = tf.placeholder(tf.float32, shape=[None, height, width, channels], name="X")
training = tf.placeholder_with_default(False, shape=[])

with slim.arg_scope(mobilenet_v2.training_scope()): #is_training=False
  logits, end_points = mobilenet_v2.mobilenet(X, is_training=training)

#with slim.arg_scope(inception.inception_v3_arg_scope()):
    #logits, end_points = inception.inception_v3(X, num_classes=1001, is_training=training)

inception_saver = tf.train.Saver()


#изучить входные данные выходного уровня:
# logits.op.inputs[0].op.inputs[0]  или end_points
#end_points["PreLogits"]

#отказаться от 2-го и 3-го измерений с помощью tf.squeeze() функции
#prelogits = tf.squeeze(end_points["global_pool"], axis=[1, 2])
layer_17 = end_points["layer_17/output"]


#prelogits = end_points["global_pool"]
#добавить окончательный полностью подключенный слой поверх этого слоя:  
n_outputs = len(flower_classes)

#op(ops.expanded_conv, stride=1, num_outputs=320),
#op(slim.conv2d, stride=1, kernel_size=[1, 1], num_outputs=1280)

with tf.name_scope("new_output_layer"):
  
    layer_18 = expanded_conv(layer_17)
                 # num_outputs=320,
                 # expansion_size=conv_blocks.expand_input_by_factor(6),
                 # stride=1,
                 # rate=1,
                 # kernel_size=None,
                 # residual=True,
                 # normalizer_fn=None,
                 # project_activation_fn=tf.identity,
                 # split_projection=1,
                 # split_expansion=1,
                 # expansion_transform=None,
                 # depthwise_location='output',
                 #depthwise_channel_multiplier=1,
                 # endpoints=None,
                 # use_explicit_padding=False)
                 # padding = 'SAME')
    global_pool = mobilenet.global_pool(layer_18)
    dropout = slim.dropout(global_pool, scope='Dropout', is_training=is_training)
    flower_logits = tf.layers.dense(dropout, n_outputs, name="flower_logits")

    
    Y_proba = tf.nn.softmax(flower_logits, name="Y_proba")
    

y = tf.placeholder(tf.int32, shape=[None])

with tf.name_scope("train"):
    print("logits shape", flower_logits.get_shape().as_list(), "label shape", y.shape )
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=flower_logits, labels=y)
    #xentropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=flower_logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    batch = tf.Variable(0, trainable=False)
    

    optimizer = tf.train.AdamOptimizer()
    flower_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="flower_logits") #flower_logits
    training_op = optimizer.minimize(loss, var_list=flower_vars)
    

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(flower_logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver() 
    

    
#Разделите свой набор данных на тренировочный набор и тестовый набор. Обучите модель тренировочному набору и оцените его на тестовом наборе.
#Во-первых, мы хотим представить классы как int, а не строки:
flower_class_ids = {flower_class: index for index, flower_class in enumerate(flower_classes)}


#Будет проще перетасовать набор данных, если мы представим его как список пар filepath / class:
flower_paths_and_classes = []
for flower_class, paths in image_paths.items():
    for path in paths:
        flower_paths_and_classes.append((path, flower_class_ids[flower_class]))
        
#Затем давайте перетасовать набор данных и разделим его на обучающий набор и тестовый набор:


np.random.shuffle(flower_paths_and_classes)
a = flower_paths_and_classes
flower_paths_and_classes_train = a[:int(len(a)*0.6)]
flower_paths_and_classes_valid = a[int(len(a)*0.6):int(len(a)*0.8)]
flower_paths_and_classes_test = a[int(len(a)*0.8):]

n_iterations_per_epoch = len(flower_paths_and_classes_train) // batch_size

print("train:", len(flower_paths_and_classes_train) ,"valid:", len(flower_paths_and_classes_valid), "test", len(flower_paths_and_classes_test))

#flower_paths_and_classes
#print("flower_paths_and_classes", len(flower_paths_and_classes))
#print("flower_paths_and_classes_valid", len(flower_paths_and_classes_valid))
X_train, y_train = prepare_data(flower_paths_and_classes_train)
X_test, y_test = prepare_data(flower_paths_and_classes_test)
X_valid, y_valid = prepare_data(flower_paths_and_classes_valid)        
#n_epochs = 10000
#batch_size = 50       

#best_loss_val = np.infty
#check_interval = 100
#checks_since_last_progress = 0
#max_checks_without_progress = 20
#best_model_params = None 

#step

def shuffle_batch(X, batch_size):
  #rnd_idx = [i for i in range(len(X)) if i>=batch_n*batch_size and i<(batch_n*batch_size+batch_size)]
  #print("rnd_idx", rnd_idx)
  rnd_idx = []
  for i in range(len(X)): rnd_idx.append(i)
  for batch_idx in np.array_split(rnd_idx, batch_size):   # np.array_split делит масим на несколько массивов
      #print("batch_idx", batch_idx)  
      X_batch = X[batch_idx]
      yield X_batch      



############################## ВЫБИРАЕМ ПЕРЕМЕННЫЕ ##########################
reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="MobilenetV2") # regular expression
restore_saver = tf.train.Saver(reuse_vars) # to restore layers 1-3

h2_cache_2 = np.empty((0, layer_17.get_shape().as_list()[1]))
h2_cache_valid_2 = np.empty((0, layer_17.get_shape().as_list()[1]))

################################################################################
with tf.Session() as sess:
    init.run()
    inception_saver.restore(sess, os.path.join(MODELS_PATH, MODEL_NAME))
    
    #for batch_n in range(len(X_train)//batch_size): 
    for X_batch in shuffle_batch(X_train, batch_size):
            h2_cache_1 = sess.run(layer_17, feed_dict={X: X_batch, training: False})
            h2_cache_2 = np.concatenate((h2_cache_2, h2_cache_1))
    #for batch_n in range(len(X_train)//batch_size):
    for X_val in shuffle_batch(X_valid, batch_size):
            h2_cache_valid_1 = sess.run(layer_17, feed_dict={X: X_val, training: False})
            h2_cache_valid_2 = np.concatenate((h2_cache_valid_2, h2_cache_valid_1))

            
############################## СОЗДАЕМ КЭШ ###################################
   
  # h2_cache = prelogits.eval(feed_dict={X: X_train})
   # h2_cache_valid = sess.run(prelogits, feed_dict={X: X_valid})
################################################################################  
    
  

`use_explicit_padding` should only be used with  SAME padding.


TypeError: ignored

In [9]:
b = input_tensor.get_shape().as_list()[3]

NameError: ignored

In [0]:

import time
n_epochs = 1000
batch_size = 40
from IPython.display import clear_output
max_checks_without_progress = 1000
checks_without_progress = 0
best_loss = np.infty
with tf.Session() as sess:
 


    init.run()
    inception_saver.restore(sess, os.path.join(MODELS_PATH, MODEL_NAME))
    for var in flower_vars:
        var.initializer.run()

    t0 = time.time()

    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train))
        for rnd_indices in np.array_split(rnd_idx, len(X_train) // batch_size):
            h5_batch, y_batch = h2_cache_2[rnd_indices], y_train[rnd_indices]
            sess.run(training_op, feed_dict={prelogits: h5_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={prelogits: h2_cache_valid_2, y: y_valid})
        if loss_val < best_loss:
            save_path = saver.save(sess, "./my_mnist_model_5_to_9_five_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        if epoch % 10 == 0:
          
          
          clear_output(wait=True)
          print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
              epoch, loss_val, best_loss, acc_val * 100))
          #print(printm())

    t1 = time.time()
    print("Total training time: {:.1f}s".format(t1 - t0))

with tf.Session() as sess:
    saver.restore(sess, "./my_mnist_model_5_to_9_five_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test[:200], y: y_test[:200]})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))





990	Validation loss: 1.655289	Best loss: 0.382186	Accuracy: 85.56%
Total training time: 123.3s
INFO:tensorflow:Restoring parameters from ./my_mnist_model_5_to_9_five_frozen
Final test accuracy: 84.00%
